# Programming in Python for Data Science

# Final Project

##Submitted by : Aditi Jaswal

# Foreword

##This notebook will be showing some exploratory data analysis for the Disney dataset located https://data.world/kgarrett/disney-character-success-00-16. We will explore about the data and then find important questions. Later in the notebook, we will try to answer those questions using descriptive analysis

# About the dataset

##There are 5 datasets related to disney, i.e, movies, characters, revenue, directors and voice-actors. My main analysis is for disney movie dataset. Walt Disney Studios serves as the cornerstone upon which The Walt Disney Company was established. Over the years, this division has been responsible for the creation of a vast catalog of over 600 films, with its inaugural release, "Snow White and the Seven Dwarfs," dating back to 1937. While a significant number of these cinematic endeavors achieved great success, a few encountered less favorable outcomes. This notebook embarks on an exploration of a dataset encompassing Disney movies, aiming to delve into the factors that underlie the success of these cinematic productions.[website](  https://goodboychan.github.io/python/datacamp/data_science/machine_learning/statistics/2020/08/23/01-Disney-Movies-and-Box-Office-Success.html )

I am going to use 3 datasets, i.e,
* **disney_movies_total_gross.csv**
* **disney-director.csv**
* **disney-voice-actor.csv**

In **disney_movies_total_gross.csv** , there are 6 columns, i.e, movie_title(title of the movie), release_date(date of release for the movie), genre(drama, adventure, musical, comedy, action) MPAA_rating(PG, PG-13, R, G) total_gross(total box office collection), inflation_adjusted_gross(Actual profit made by the movie) 

In **disney-director.csv**, 2 columns,i.e Name(Name of the movie) and director(director of the movie) are there.

In **disney-voice-actor.csv**, 3 columns, character(character name whose voice is given), voice actor(name of the voice actor) and movie(Name of the movie)are there. 


# Questions of Interest

#### In this notebook, I will be finding top 10 movies which did best business on box office, best movie from each genre. Also, I will be exploring if MPAA_rating impacted the popularity of movies or not. 
#### Also, I am going to check the popularity trends for all 5 genre in disney movies. 
#### For, director and voice datasets, I am going to merge them in one dataset and explore which director used maximum number of voice artists, and the movie which used maximum number of voice artists

# Import libraries needed for this lab
import pandas as pd
import random
import test_assignment6 as t
import altair as alt
import string
import inspect
from hashlib import sha1

In [ ]:
# Import libraries needed for this lab
import pandas as pd
import random
import altair as alt
import string
import inspect
from hashlib import sha1
import matplotlib.pyplot as plt


In [ ]:
movies = pd.read_csv("data/disney_movies_total_gross.csv")

In [ ]:
movies.head()

In [ ]:
movies.describe()

In [ ]:
movies.info()

## Here we see that datatype of all the column is object. Lets convert them to correct datatype so we can perform analysis

In [ ]:
# Remove dollar signs and commas from the "inflation_adjusted_gross" column
movies['inflation_adjusted_gross'] = movies['inflation_adjusted_gross'].str.replace('$', '').str.replace(',', '')

In [ ]:
#Convert the "inflation_adjusted_gross" column to integer
movies['inflation_adjusted_gross'] = movies['inflation_adjusted_gross'].astype(int)

In [ ]:
movies['total_gross'] = movies['total_gross'].str.replace('$', '').str.replace(',', '')

In [ ]:
#Convert the "total_gross" column to integer
movies['total_gross'] = movies['total_gross'].astype(int)

In [ ]:
movies['release_date'] = pd.to_datetime(movies['release_date'])

## lets find out top 10 movies which did the best business in the market:

In [ ]:
top_movies = movies.sort_values('inflation_adjusted_gross', ascending=False)

# Display the top 10 movies 
top_movies.head(10)

## now lets see the best movie from each genre

In [ ]:
genre_wise_best= movies.groupby("genre")['movie_title', 'inflation_adjusted_gross'].max()
genre_wise_best

##lets explore MPAA_rating column. From describe(), we can see that there are 5 unique entries for this column, i.e, PG, G, PG-13, R and Not rated. From https://en.wikipedia.org/wiki/Motion_Picture_Association_film_rating_system , PG = Parentl Guidance, G - General Audience, PG-13 = Parents Strongly Cautioned and R = Restricted. 

In [ ]:
rating_counts = movies['MPAA_rating'].value_counts()
rating_counts

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(rating_counts.index, rating_counts)
plt.title('Number of Movies per MPAA Rating')
plt.xlabel('MPAA Rating')
plt.ylabel('Number of Movies')
plt.xticks(rotation=45)
plt.show()

## Lets find out if there is any trend between the ,MP rating and business done by disney movies:

In [ ]:
average_gross = movies.groupby('MPAA_rating')['inflation_adjusted_gross'].mean()

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(average_gross.index, average_gross)
plt.title('Average Inflation Adjusted Gross per MPAA Rating')
plt.xlabel('MPAA Rating')
plt.ylabel('Average Inflation Adjusted Gross')
plt.xticks(rotation=45)
plt.show()

##here we found out that although least number of movies were 'not rated' but those movies did maximum of the business.However, movies made for general audiences also did good business.

## From the above stats, it seems that some genre are more polpular than others. lets find out movie genre trend

In [ ]:
## finding release year from release date column
movies['release_year'] = movies['release_date'].dt.year

In [ ]:


# Group the movies data by 'genre' and 'release_year', and compute the mean 'adjusted_gross' for each group
grouped_movies = movies.groupby(['genre', 'release_year'])['inflation_adjusted_gross'].mean().reset_index()

# Display the first 10 rows of the grouped_movies DataFrame
print(grouped_movies.head(10))

In [ ]:
# Create a plot for the genre with the highest mean adjusted gross in each year
plt.figure(figsize=(12, 6))
for genre, df in grouped_movies.groupby('genre'):
    df.plot(x='release_year', y='inflation_adjusted_gross', label=genre)
plt.title('Box Office Revenues of Movies Grouped by Genre and Release Year')
plt.xlabel('Release Year')
plt.ylabel('Adjusted Gross (in millions)')
plt.legend()
plt.show()

In [ ]:

grouped_data = grouped_movies.groupby('genre')

# Plot the data
for genre, genre_data in grouped_data:
    plt.plot(genre_data['release_year'], genre_data['inflation_adjusted_gross'], label=genre)

# Set up the plot
plt.title('Box Office Revenues of Movies Grouped by Genre and Release Year')
plt.xlabel('Release Year')
plt.ylabel('Adjusted Gross (in millions)')
plt.legend()
plt.show()

##From the above line plot, we can see that action genre was growing the fastest. However musical dropped significantly. Thus, disney's movie genre were changing in terms of popularity over the period of time

## lets explore other 2 datasets now

In [ ]:
director = pd.read_csv("data/disney-director.csv")

In [ ]:
director

In [ ]:
voice = pd.read_csv("data/disney-voice-actors.csv")

In [ ]:
voice

## lets create a function named "merge_dataframes_inner " to merge 2 datsets and merge datasets director and voice in one dataset named voice_director

In [ ]:


def merge_dataframes_inner(df1, df2, common_column):
    """
    Merge two DataFrames using an inner join on a common column.

    Parameters
    ----------
    df1: DataFrame
      The first DataFrame to be merged.
    df2: DataFrame
      The second DataFrame to be merged.
    common_column: str
      The name of the common column to perform the inner join.

    Returns
    ----------
    merged_df: DataFrame
      The resulting DataFrame after merging the two input DataFrames using an inner join on the common column.
      
    Examples
    ----------
    
    df1 = pd.DataFrame({
    'ID': [1, 2, 3, 4],
    'Name': ['Alice', 'Bob', 'Charlie', 'David']
})

    df2 = pd.DataFrame({
    'ID': [3, 4, 5, 6],
    'Age': [25, 30, 22, 28]
})
    """
    merged_df = pd.merge(df1, df2, on=common_column, how='inner')
    return merged_df



In [ ]:
# Rename the 'name' column in the 'director' dataframe to 'movie' for a consistent column name
director = director.rename(columns={'name': 'movie'})

#Calling the function to merge the datasets
voice_director = merge_dataframes_inner(director, voice, 'movie')
voice_director



## unit testing

In [ ]:
# Helper DataFrames for unit tests
df1 = pd.DataFrame({
    'ID': [1, 2, 3, 4],
    'Name': ['Alice', 'Bob', 'Charlie', 'David']
})

df2 = pd.DataFrame({
    'ID': [3, 4, 5, 6],
    'Age': [25, 30, 22, 28]
})

# Unit test 1: Test merging on 'ID'
merged_result = merge_dataframes_inner(df1, df2, 'ID')
expected_result = pd.DataFrame({
    'ID': [3, 4],
    'Name': ['Charlie', 'David'],
    'Age': [22, 28]
})
merged_result.shape == expected_result.shape




In [ ]:
voice_director.describe()

In [ ]:
voice_director.isnull()

In [ ]:
voice_director.head()

## Lets find out which director wortked with maximum number of voice-actors in their movie

In [ ]:


## using chaining for group by director and then get the unique values for voice-actors who worked with the director
director_voice_actor_count = voice_director.groupby('director')['voice-actor'].nunique().reset_index(name='voice_actor_count')
director_voice_actor_count = director_voice_actor_count.sort_values(by='voice_actor_count', ascending=False)

print(director_voice_actor_count)

## Lets find out the movie which used max numbers of voice artists

In [ ]:
movie_voice_actor_count = voice.groupby('movie')['voice-actor'].nunique().reset_index(name='voice_actor_count')
movie_voice_actor_count = movie_voice_actor_count.sort_values(by='voice_actor_count', ascending=False)

print(movie_voice_actor_count)

##Who directed the movie with maximum number of voice artists in it? What was the name of the movie and what was the count of voice actor?

## Ques: Find out the name of movie in which maximum number of voice-actors were used? Also find out the count of voice actors and the director who directed that movie? 

In [ ]:


# Create a new column to count the number of voice actors per movie
voice_director['voice_actor_count'] = voice_director.groupby('movie')['voice-actor'].transform('count')

# Filter the dataset to include only the rows of the movie with the maximum number of voice actors
max_voice_actors_movie = voice_director[voice_director['voice_actor_count'] == voice_director['voice_actor_count'].max()]

# Print the movie details
print(max_voice_actors_movie[['movie', 'director', 'voice_actor_count']].drop_duplicates())


#lets find out top 10 movies

## using black for improving function

In [ ]:
!black my_functions.py

In [ ]:
!black test_my_function.py

## using pytest

In [ ]:
import pytest

In [ ]:
pytest

In [ ]:
!pytest test_my_function.py

### Thank you :)